# AoC 2023, Day 1, Problem 1

--- Day 3: Gear Ratios ---

You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:

```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

## Download Input
- https://adventofcode.com/2023/day/3/input
- select all, CMD+C
- open TextEdit, Make Plain Text
- CMD+V, save as input.txt

In [5]:
# This function can be used for every puzzle to retrieve input
def get_input(filename="input.txt"):
    with open(filename, "r") as f:
        return f.read().strip()

# Let's save the input into the variable data
data = get_input()
# Split lines and make a list of strings
data_lines = data.splitlines()
data_lines[:10]

['..224.....487...................718.....................378............................................284........310......313..........311.',
 '....*..............................*744....486*485......*......741......@...359.#666...439................*925....*......$..+........@......',
 '.235................758..440...........................251....*......262.....*..........*......................752......774.......515.......',
 '.........705%..@746........+..942*591.347.470...#..257.........637...........793.......299..../.....813....509......464......&.........688..',
 '.....82................................*.../..901.....*..................836.....&............814...*........*..............80...17*....*...',
 '.../...*...679.661.....299...........222.............875.....213...161............964...894.........998.....310....258.85...........735.586.',
 '.650..23..#......*.......................................760*........@./........................202...................*.....339.

## Devise a Solution
All right. Let's start thinking about a solution. When I look at this example, I visually start from left to right. Whenever I encounter a number, I inspect the character to the right, the char to the left. Then according to the lenght of the number (for example 35 has length 2) I inspect the same numbers of characters above and below. Then I take the first and last digit and respectively I look at the diagonal top left, bottom left and top right bottom right.

what I need:

- split dataset into lines
- iterate using indexes so that I can compare to indexes of previous and next lines if they exist
- then iterate through characters
- I need a way to identify a number. perhaps I can use isdigit()
- for each digit check top, bottom, left, right, diagonals. if any of these contains a symbol, identify the full number and add the number up
- if char is digit, save char into temp string and do so until you have the full number
- I need a way to identify symbols and exclude dots (I have no idea)

In [8]:
# For now let's work only on the sample
samp = '''467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..'''

lines = samp.splitlines()
lines

['467..114..',
 '...*......',
 '..35..633.',
 '......#...',
 '617*......',
 '.....+.58.',
 '..592.....',
 '......755.',
 '...$.*....',
 '.664.598..']

In [10]:
# Let's first work on the first line to find the numbers and their final indexes.
x = lines[8]
print(x)
# list of tuples where to append number, start and end indexes
num_list = []
temp_num = ''
for i in range(len(x)):
    if x[i].isdigit():
        if temp_num == '':
            start_index = i
        temp_num+=x[i]
    else:
        if temp_num != '':
            end_index = i-1
            num_list.append((temp_num, start_index, end_index))
        temp_num = ''
# append number to list in case we are at the end of the loop
if temp_num != '':
    end_index = len(x)-1
    num_list.append((temp_num, start_index, end_index))
num_list

...$.*....


[]

In [12]:
# I will now create a function to make things easier 
def get_numbers(line):
    num_list = []
    temp_num = ''
    for i in range(len(line)):
        if line[i].isdigit():
            if temp_num == '':
                start_index = i
            temp_num+=line[i]
        else:
            if temp_num != '':
                end_index = i-1
                num_list.append((temp_num, start_index, end_index))
            temp_num = ''
        # append number to list in case we are at the end of the loop
    if temp_num != '':
        end_index = len(line)-1
        num_list.append((temp_num, start_index, end_index))
    return num_list
get_numbers(lines[0])

[('467', 0, 2), ('114', 5, 7)]

In [14]:
# this function check if a char is a symbol and not a point and returns true or false
def is_symbol(char):
    return char != '.' and not char.isdigit()

# let's check if for each line the numbers have a symbol in any of the indexes to check.
def get_solution(lines):
    total_sum = 0
    row_index = 0

    for line in lines:
        numbers = get_numbers(line)

        for o in numbers:
            send_num = False
            num_len = len(o[0])
            # check line above for symbols
            start_index = o[1]
            # check top left diagonal
            if row_index-1>=0 and o[1]-1>=0:
                if is_symbol(lines[row_index-1][o[1]-1]):
                    send_num = True
            # check bottom left diagonal
            if row_index+1<len(lines)-1 and o[1]-1>=0:
                if is_symbol(lines[row_index+1][o[1]-1]):
                    send_num = True
            # check left char
            if o[1]-1>=0:
                if is_symbol(lines[row_index][o[1]-1]):
                    send_num = True
            # check top right diagonal
            if row_index-1>=0 and o[2]+1 < len(lines[row_index-1]):
                if is_symbol(lines[row_index-1][o[2]+1]):
                    send_num = True
            # check bottom right diagonal
            if row_index+1 < len(lines)-1 and o[2]+1<len(lines[row_index+1]):
                if is_symbol(lines[row_index+1][o[2]+1]):
                    send_num = True
            # check right char
            if o[2]+1 < len(lines[row_index]):
                if is_symbol(lines[row_index][o[2]+1]):
                    send_num = True
            # check top row
            if row_index-1>=0:
                for i in range(o[1],o[2]+1):
                    if is_symbol(lines[row_index-1][i]):
                        send_num = True
            # check bottom row
            if row_index+1 < len(lines):
                for i in range(o[1],o[2]+1):
                    if is_symbol(lines[row_index+1][i]):
                        send_num = True
            if send_num == True:
                total_sum+=int(o[0])
        row_index+=1

    return total_sum

get_solution(lines)

4361

In [16]:
get_solution(data_lines)

539590

# Problem 2



## Devise a Solution
- c